In [1]:
pip install mtcnn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.5 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import matplotlib.pyplot as plt
import mtcnn
from mtcnn.mtcnn import MTCNN
from matplotlib.patches import Rectangle
from matplotlib.patches import Circle
import torchvision.transforms as transforms
import json

In [3]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import numpy as np
import shutil
import random
from torchvision.utils import make_grid
import torch.nn.functional as F
from tqdm import tqdm
import cv2

In [4]:
root_path = '/kaggle/working'
data_dir = "/kaggle/input/deepfake/DFWILD/train_real"
# data_dir = "/kaggle/input/test-mtcnn"

In [5]:
def Resize_Image():
    # Open the image with alpha channel
    image = Image.open("/tmp/temp_cropped_image.png")
    
    # Remove the alpha channel (only keep RGB channels)
    image_rgb = image.convert("RGB")
    
    # Resize the image to (256, 256)
    image_resized = image_rgb.resize((256, 256))
    
    # # Convert to numpy array for displaying
    # image_resized_np = np.array(image_resized)
    
    # plt.axis("off")
    # plt.imshow(image_resized_np)

    return image_resized

In [6]:
# draw an image with detected objects
def draw_facebox(imagename, filename, result_list):

    if (len(result_list) == 0):
        return
        
    result = result_list[0]
    # load the image
    data = plt.imread(filename)
    # print(data.shape)

    # get coordinates
    x, y, width, height = result['box']
    
    # Save or display the cropped image
    fig, ax = plt.subplots()
    
    # plot the image
    ax.imshow(data)
    
    # Make a random vector with 1s and 0s
    rand_bits = np.random.choice(2,len(result['keypoints'].items()))
    idx = 0
    
    # draw the dots
    for key, value in result['keypoints'].items():
    # create and draw dot
        if (rand_bits[idx]):
            dot = plt.Circle(value, radius=20, color='black')
            ax.add_patch(dot)
        idx += 1

    rectL = plt.Rectangle((0, 0), x, 256, fill=True, color='black')
    rectT = plt.Rectangle((0, 0), 256, y, fill=True, color='black')
    rectR = plt.Rectangle((x+width, 0), (256-x-width), 256, fill=True, color='black')
    rectB = plt.Rectangle((0, y+height), 256, (256-y-height), fill=True, color='black')
    
    # draw the box
    ax.add_patch(rectL)
    ax.add_patch(rectT)
    ax.add_patch(rectR)
    ax.add_patch(rectB)

    rect = plt.Rectangle((x, y), width, height,fill=False, color='orange')
    # draw the box
    ax.add_patch(rect)
    
    # Turn off the axis
    ax.axis("off")

    # Save the cropped image temporarily
    temp_cropped_path = "/tmp/temp_cropped_image.png"
    fig.savefig(temp_cropped_path, bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    
    # Open the cropped image using PIL
    image_resized = Resize_Image()

    # Save only ax[1] (the cropped image with annotations)
    save_path = "/kaggle/working/cropped_images"

    if not os.path.exists(save_path):
        os.makedirs(save_path)

    output_file = os.path.join(save_path, imagename)

    image_resized.save(output_file)
    
    # show the plot
    # plt.show()

In [ ]:
results = {}
real_images = [f for f in os.listdir(data_dir) if f.endswith((".png", ".jpg", ".jpeg"))]

idx = 0

# detector initialization
detector = mtcnn.MTCNN()

save_path = "/kaggle/working/cropped_images"
if os.path.exists(save_path):
    shutil.rmtree(save_path)

for imagename in real_images:
    idx += 1
    
    filename = os.path.join(data_dir, imagename) #File location
    image = Image.open(filename)
    
    # Find coordinates
    pixels = np.asarray(image)
    # detect faces in the image
    faces = detector.detect_faces(pixels)

    # Convert NumPy types to Python types
    results[imagename] = [
        {
            "box": [int(coord) for coord in face['box']],
            "confidence": float(face['confidence']),
            "keypoints": {key: [int(coord) for coord in value] for key, value in face['keypoints'].items()}
        }
        for face in faces
    ]

    # display faces on the original image
    # if (idx == 1):
    #     draw_facebox(imagename, filename, faces)
    #     idx = 0

    # Save results to a JSON file
    output_file = "/kaggle/working/faces_data.json"
    with open(output_file, "w") as f:
        json.dump(results, f, indent=4)
    
    print(f"Results saved to {output_file}")

Results saved to /kaggle/working/faces_data.json
Results saved to /kaggle/working/faces_data.json
Results saved to /kaggle/working/faces_data.json
Results saved to /kaggle/working/faces_data.json
Results saved to /kaggle/working/faces_data.json
Results saved to /kaggle/working/faces_data.json
Results saved to /kaggle/working/faces_data.json
Results saved to /kaggle/working/faces_data.json
Results saved to /kaggle/working/faces_data.json
Results saved to /kaggle/working/faces_data.json
Results saved to /kaggle/working/faces_data.json
Results saved to /kaggle/working/faces_data.json
Results saved to /kaggle/working/faces_data.json
Results saved to /kaggle/working/faces_data.json
Results saved to /kaggle/working/faces_data.json
Results saved to /kaggle/working/faces_data.json
Results saved to /kaggle/working/faces_data.json
Results saved to /kaggle/working/faces_data.json
Results saved to /kaggle/working/faces_data.json
Results saved to /kaggle/working/faces_data.json
Results saved to /ka

In [ ]:
image = Image.open("/kaggle/working/cropped_images/train_real_0000048.png")
image = np.array(image)
plt.imshow(image)
plt.show()